In [2]:
import pandas as pd
import numpy as np 
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense

2023-05-10 13:38:19.744041: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
data = pd.read_csv('dataset.csv')
data

,timer,fight_result,has_round_started,is_round_over,player1.player_id,player1.health,player1.is_crouching,player1.is_jumping,player1.is_player_in_move,player1.move_id,...,player2.player_buttons.right,player2.player_buttons.left,player2.player_buttons.select,player2.player_buttons.start,player2.player_buttons.Y,player2.player_buttons.B,player2.player_buttons.X,player2.player_buttons.A,player2.player_buttons.L,player2.player_buttons.R
0,151,NOT_OVER,True,False,0,144,False,False,False,0,...,False,False,False,False,False,False,False,False,False,False
1,151,NOT_OVER,True,False,0,144,False,False,False,0,...,False,False,False,False,False,False,False,False,False,False
2,151,NOT_OVER,True,False,0,144,False,False,False,0,...,False,False,False,False,False,False,False,False,False,False
3,151,NOT_OVER,True,False,0,144,False,False,False,0,...,False,False,False,False,False,False,False,False,False,False
4,151,NOT_OVER,True,False,0,144,False,False,False,0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15005,133,NOT_OVER,True,False,0,176,False,False,True,1,...,False,False,False,False,False,False,False,False,False,False
15006,133,NOT_OVER,True,False,0,176,False,False,True,1,...,False,False,False,False,False,False,False,False,False,False
15007,133,NOT_OVER,True,False,0,176,False,False,True,1,...,False,False,False,False,False,False,False,False,False,False
15008,133,NOT_OVER,True,False,0,176,False,False,True,1,...,False,False,False,False,False,False,False,False,False,False


In [4]:
round = 1
round_list = []
flag = 0
for i in range(data.shape[0]):
    d = data.iloc[i][3]
    if (d == True):
        flag = 1
    round_list.append(round)
    if flag == 1:
        round = round + 1
        flag = 0

data['round'] = round_list

In [5]:
rounds_where_P1_won = data[data['fight_result'] == 'P1']['round'].values

In [6]:
#for i in range(len(rounds_where_P1_won)):
df = data[data['round'] == rounds_where_P1_won[1]]
df_2 = df[df['player1.is_player_in_move'] == True]
df_3 = df[df['player2.is_player_in_move'] == True]
df_2 = df_2.drop(columns=['timer','fight_result','has_round_started','is_round_over','player1.x_coord', 'round','player1.y_coord', 'player1.player_id','player1.health','player1.is_player_in_move','player1.move_id','player1.player_buttons.select','player1.player_buttons.start','player2.player_id','player2.health','player2.is_crouching','player2.is_jumping','player2.is_player_in_move','player2.move_id','player2.x_coord','player2.y_coord','player2.player_buttons.up','player2.player_buttons.down','player2.player_buttons.right','player2.player_buttons.left','player2.player_buttons.select','player2.player_buttons.start','player2.player_buttons.Y','player2.player_buttons.B','player2.player_buttons.X','player2.player_buttons.A','player2.player_buttons.L','player2.player_buttons.R'])
#df_3 = df_3.drop(columns=['timer','fight_result','has_round_started','is_round_over','player1.player_id','player1.health','player1.is_crouching','player1.is_jumping','player1.is_player_in_move','player1.move_id','player1.x_coord','player1.y_coord','player1.player_buttons.up','player1.player_buttons.down','player1.player_buttons.right','player1.player_buttons.left','player1.player_buttons.select','player1.player_buttons.start','player1.player_buttons.Y','player1.player_buttons.B','player1.player_buttons.X','player1.player_buttons.A','player1.player_buttons.L','player1.player_buttons.R','player2.player_id','player2.health','player2.is_player_in_move','player2.move_id','player2.player_buttons.up','player2.player_buttons.down','player2.player_buttons.right','player2.player_buttons.left','player2.player_buttons.select','player2.player_buttons.start','player2.player_buttons.Y','player2.player_buttons.B','player2.player_buttons.X','player2.player_buttons.A','player2.player_buttons.L','player2.player_buttons.R'])


In [28]:
df_2.values[0]

array([False, False, False, False,  True, False,  True, False, False,
       False, False, False])

In [22]:
# Define the autoencoder architecture
input_shape = (12,)
latent_dim = 4

input_layer = Input(shape=input_shape)
encoded = Dense(latent_dim, activation='relu')(input_layer)
decoded = Dense(input_shape[0], activation='sigmoid')(encoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

encoder = Model(input_layer, encoded)
decoder_input = Input(shape=(latent_dim,))
decoder_output = autoencoder.layers[-1](decoder_input)
decoder = Model(decoder_input, decoder_output)


In [9]:
for i in range(len(rounds_where_P1_won)):
    df = data[data['round'] == rounds_where_P1_won[1]]
    df_2 = df[df['player1.is_player_in_move'] == True]
    df_2 = df_2.drop(columns=['timer','fight_result','has_round_started','is_round_over','player1.x_coord', 'round','player1.y_coord', 'player1.player_id','player1.health','player1.is_player_in_move','player1.move_id','player1.player_buttons.select','player1.player_buttons.start','player2.player_id','player2.health','player2.is_crouching','player2.is_jumping','player2.is_player_in_move','player2.move_id','player2.x_coord','player2.y_coord','player2.player_buttons.up','player2.player_buttons.down','player2.player_buttons.right','player2.player_buttons.left','player2.player_buttons.select','player2.player_buttons.start','player2.player_buttons.Y','player2.player_buttons.B','player2.player_buttons.X','player2.player_buttons.A','player2.player_buttons.L','player2.player_buttons.R'])
    autoencoder.fit(df_2.values, df_2.values, epochs=100, batch_size=32)
    


Epoch 1/100
30/30 [==============================] - 1s 8ms/step - loss: 0.2532
Epoch 2/100
30/30 [==============================] - 0s 8ms/step - loss: 0.2421
Epoch 3/100
30/30 [==============================] - 0s 11ms/step - loss: 0.2313
Epoch 4/100
30/30 [==============================] - 0s 13ms/step - loss: 0.2205
Epoch 5/100
30/30 [==============================] - 0s 8ms/step - loss: 0.2092
Epoch 6/100
30/30 [==============================] - 0s 7ms/step - loss: 0.1976
Epoch 7/100
30/30 [==============================] - 0s 6ms/step - loss: 0.1857
Epoch 8/100
30/30 [==============================] - 0s 16ms/step - loss: 0.1731
Epoch 9/100
30/30 [==============================] - 0s 10ms/step - loss: 0.1599
Epoch 10/100
30/30 [==============================] - 0s 8ms/step - loss: 0.1483
Epoch 11/100
30/30 [==============================] - 0s 10ms/step - loss: 0.1379
Epoch 12/100
30/30 [==============================] - 0s 8ms/step - loss: 0.1291
Epoch 13/100
30/30 [============

In [10]:

#autoencoder.fit(input_data, input_data, epochs=100, batch_size=32)

latent_points = np.random.normal(size=(1000, latent_dim))
generated_data = decoder.predict(latent_points)
print(generated_data)


32/32 [==============================] - 0s 7ms/step
[[7.6122177e-01 5.3719466e-04 8.6827686e-06 ... 1.3137949e-04
  9.7229797e-04 7.3257947e-01]
 [9.9999517e-01 4.7440448e-05 5.1682728e-06 ... 6.3446641e-02
  2.2927865e-01 9.9996179e-01]
 [9.9998921e-01 2.5944307e-04 2.0479809e-02 ... 4.5393175e-01
  4.6628523e-01 9.1752529e-01]
 ...
 [9.9973541e-01 8.4571034e-04 8.7282068e-01 ... 7.0441061e-01
  5.0433117e-01 4.9621530e-02]
 [9.9992853e-01 3.5599156e-04 1.9283925e-01 ... 5.7103235e-01
  4.0449470e-01 4.7148418e-01]
 [9.2814392e-01 2.1735200e-04 8.7895123e-03 ... 5.5024005e-02
  8.1370428e-02 9.5539808e-01]]


In [11]:
for i in range(len(generated_data)):
    for j in range(len(generated_data[i])):
        if generated_data[i][j] < 0.5:
            generated_data[i][j] = False
            
        else:
            generated_data[i][j] = True

In [12]:
generated_data = generated_data.astype(bool)
generated_data

array([[ True, False, False, ..., False, False,  True],
       [ True, False, False, ..., False, False,  True],
       [ True, False, False, ..., False, False,  True],
       ...,
       [ True, False,  True, ...,  True,  True, False],
       [ True, False, False, ...,  True, False, False],
       [ True, False, False, ..., False, False,  True]])

In [13]:
generated_data

array([[ True, False, False, ..., False, False,  True],
       [ True, False, False, ..., False, False,  True],
       [ True, False, False, ..., False, False,  True],
       ...,
       [ True, False,  True, ...,  True,  True, False],
       [ True, False, False, ...,  True, False, False],
       [ True, False, False, ..., False, False,  True]])

In [14]:
np.savetxt("generated_data.csv", generated_data.astype(int), delimiter=",", fmt="%d")